# Inference with Own Data
- Take a pre-trained device-specific machine learning model and feed it with aggregated smart-meter data.
- Inference happens in batches of 480 data points. If the aggregated data does not align with this window, the last excess data points can be dropped.
- The model returns a DataFrame which is a guess which part of the aggregated data was caused by the device.
- The inferred data is normalized to a range between [0, 1].

In [1]:
from pyprojroot import here

from electricity.inference_with_own_data import inference

In [2]:
config = {
    # Aggregated data
    "data_file": here("data/test/geert/main.csv"),

    # Saved model from previous training
    "model_file": here("results_UK-DALE_TitanV_kettle/uk_dale/kettle/best_acc_model.pth"),

    # Inference device
    "device": "cpu",

    # Re-sample data if necessary
    "sampling": "6s",

    # Keep aggregated data below this threshold
    "cutoff": [2000],
}

In [3]:
inference_data = inference(config)

297.5686214145706
102.7753549362568


## Visualize Raw and Infered Data

In [4]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

### Infered Data

In [5]:
fig = px.line(inference_data, line_shape="linear", render_mode="svg")
fig.show()

### Ground Truth

In [6]:
kettle = pd.read_csv("./data/test/geert/kettle.csv", sep=',', header=None, names=["time", "kettle"])
kettle["time"] = pd.to_datetime(kettle["time"])
kettle = kettle.set_index("time")

fig = px.line(kettle, line_shape="linear", render_mode="svg")
fig.show()

### Aggregated Data for Reference

In [7]:
main = pd.read_csv(config["data_file"], sep=',', header=None, names=["time", "aggregate"])
main["time"] = pd.to_datetime(main["time"])
main = main.set_index("time")

fig = px.line(main, line_shape="linear", render_mode="svg")
fig.show()

In [8]:
import numpy as np
print(np.mean(kettle))
print()
print(np.std(kettle))

10.706187987160565

kettle    141.631825
dtype: float64
